# DSE 309 | Final Project
## Project Title: ALGORITHMIC TRADING USING PYTHON
## Submitted By:                                
### 1) Jayesh Kumpawat | 19148
### 2) Shubhajit Dey | 19299


### Importing Libraries

In [1]:
import numpy as np  
import pandas as pd  
import requests  
import xlsxwriter  
import math 
from scipy import stats 

* In this project, we analyse the 500 largest companies of US as per the S&P 500 Index Funds.
* A .csv file containing the name, symbol and the sector of companies is read using pandas.

In [2]:
stocks = pd.read_csv('S&P 500-stocks.csv')
stocks

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


* We are using the `IEX Cloud API(Sandbox version)` for getting the data related to the stocks. A unique API token is obtained after registering on the website which needs to be used.
* The `requests` library is used to get the data
* Using the json method, we obtain the data in the form of a dictionary.

In [3]:
symbol = 'TSLA'
IEX_API_TOKEN = 'Tpk_8b62f9d4f8b74dc1989fd7e2a265afb2'
url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_API_TOKEN}'
r = requests.get(url, verify = False)
data = r.json()
data

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'avgTotalVolume': 30027443,
 'calculationPrice': 'close',
 'change': -6.19,
 'changePercent': -0.00588,
 'close': 1058.16,
 'closeSource': 'iaicfofl',
 'closeTime': 1706575579157,
 'companyName': 'Tesla Inc',
 'currency': 'USD',
 'delayedPrice': 1056.15,
 'delayedPriceTime': 1654368220302,
 'extendedChange': 2.07,
 'extendedChangePercent': 0.00198,
 'extendedPrice': 1026,
 'extendedPriceTime': 1712477269967,
 'high': 1066.01,
 'highSource': 'dcemel1r eypu  en5tidai',
 'highTime': 1704674537292,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 1034.46,
 'iexCloseTime': 1674670719935,
 'iexLastUpdated': 1653640155346,
 'iexMarketPercent': 0.020003353117424016,
 'iexOpen': 1012.57,
 'iexOpenTime': 1668413704543,
 'iexRealtimePrice': 1018.25,
 'iexRealtimeSize': 5,
 'iexVolume': 558434,
 'lastTradeTime': 1677598667297,
 'latestPrice': 1012.93,
 'latestSource': 'Close',
 'latestTime': 'December 6, 2021',
 'latestUpdate': 1705314398857,
 'latestVolume':

Let us try to print some of the features of the company- Tesla Inc. :

In [4]:
print(data['latestPrice'])  
print(data['marketCap'])

1012.93
1046728457783


### API Call in Batches

Now, the data needs to be called for all the stocks. If this is done one symbol(stock) at a time; then the time consumption of this step is way high!


Therefore, the request for data is given in batches of 100 using the `chunks` function:

In [5]:
# Function sourced from: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):    # Gives n-sized 
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
sym_groups = list(chunks(stocks['Symbol'], 100))
print(sym_groups)

[0      MMM
1      AOS
2      ABT
3     ABBV
4     ABMD
      ... 
95      CE
96     CNC
97     CNP
98    CDAY
99    CERN
Name: Symbol, Length: 100, dtype: object, 100      CF
101     CRL
102    SCHW
103    CHTR
104     CVX
       ... 
195     FRC
196      FE
197    FISV
198     FLT
199     FMC
Name: Symbol, Length: 100, dtype: object, 200       F
201    FTNT
202     FTV
203    FBHS
204    FOXA
       ... 
295     MRO
296     MPC
297    MKTX
298     MAR
299     MMC
Name: Symbol, Length: 100, dtype: object, 300     MLM
301     MAS
302      MA
303    MTCH
304     MKC
       ... 
395      RF
396     RSG
397     RMD
398     RHI
399     ROK
Name: Symbol, Length: 100, dtype: object, 400     ROL
401     ROP
402    ROST
403     RCL
404    SPGI
       ... 
495    WLTW
496    WYNN
497     XEL
498    XLNX
499     XYL
Name: Symbol, Length: 100, dtype: object, 500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Name: Symbol, dtype: object]


It can be seen that the data becomes a list of lists

In [6]:
print(len(sym_groups))

6


In [7]:
sym_strings = []
for i in range(len(sym_groups)):
    sym_strings.append(','.join(sym_groups[i]))
    print(sym_strings[i])

MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN
CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCA,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC
F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUM,HBAN,HII

Now, we have a list of 6 elements where each element consists of symbols separated by commas:

In [8]:
sym_strings

['MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,ANTM,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY,CERN',
 'CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCA,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC',
 'F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HU

In [9]:
len(sym_strings)

6

Now, it is time to move on to employing strategies for having the best investment:

# 1) Quantitative Momentum Strategy


Momentum investing is a trading strategy in which investors buy securities that are rising and sell them when they look to have peaked.
The strategy is to filter out highest momentum stocks i.e., stocks which are stable and perform well in long periods of time. 

* High-quality momentum stocks perform slow and steady over long periods of time.
* Low-quality momentum stocks are ones that does not show any momentum for a long time.

High-quality momentum stocks are more preferred because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future.


In [10]:
# Creating a list for the columns of the dataframe .
HQM_cols = [
                'Symbol', 
                'Price', 
                'Quantity of Shares to Buy', 
                '1-Year Price Return', 
                '1-Year Return Percentile',
                '6-Month Price Return',
                '6-Month Return Percentile',
                '3-Month Price Return',
                '3-Month Return Percentile',
                '1-Month Price Return',
                '1-Month Return Percentile',
                'HQM Score'
                ]

In [11]:
HQM_df = pd.DataFrame(columns = HQM_cols)

In [12]:
HQM_df

,Symbol,Price,Quantity of Shares to Buy,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile,HQM Score


In [13]:
for sym_string in sym_strings:                 # Looping through each symbols in a batch of 100
    batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={sym_string}&token={IEX_API_TOKEN}'
    data1 = requests.get(batch_url, verify = False).json()
    
    for symbol in sym_string.split(','):      # Splitting the strings on commas and feeding data accordingly row by row
        HQM_df = HQM_df.append(
                                pd.Series([
                                           symbol, 
                                           data1[symbol]['quote']['latestPrice'],
                                           'NA',
                                           data1[symbol]['stats']['year1ChangePercent'],
                                           'NA',
                                           data1[symbol]['stats']['month6ChangePercent'],
                                           'NA',
                                           data1[symbol]['stats']['month3ChangePercent'],
                                           'NA',
                                           data1[symbol]['stats']['month1ChangePercent'],
                                           'NA',
                                           'NA'
                                            ], 
                                           index = HQM_cols), 
                                ignore_index = True
                              )
        
HQM_df

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/

,Symbol,Price,Quantity of Shares to Buy,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile,HQM Score
0,MMM,178.80,NA,0.033379,NA,-0.153044,NA,-0.109283,NA,-0.044634,NA,NA
1,AOS,84.00,NA,0.517625,NA,0.195715,NA,0.120902,NA,0.052388,NA,NA
2,ABT,134.81,NA,0.232279,NA,0.206052,NA,0.014829,NA,0.048656,NA,NA
3,ABBV,126.20,NA,0.164770,NA,0.085258,NA,0.079906,NA,0.014847,NA,NA
4,ABMD,315.86,NA,0.104620,NA,0.040211,NA,-0.198809,NA,-0.172282,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.33,NA,0.206232,NA,0.060442,NA,-0.042776,NA,-0.011133,NA,NA
501,ZBRA,588.85,NA,0.580647,NA,0.145240,NA,-0.004040,NA,-0.033181,NA,NA
502,ZBH,129.86,NA,-0.190522,NA,-0.241164,NA,-0.176125,NA,-0.146554,NA,NA
503,ZION,65.85,NA,0.459487,NA,0.080987,NA,0.092121,NA,-0.040920,NA,NA


### Momentum Percentiles

Calculating the percentiles of return on different periods taking help of the `SciPy` module:

In [14]:
from scipy.stats import percentileofscore as ptile

t_periods = [
                '1-Year',
                '6-Month',
                '3-Month',
                '1-Month'
            ]

for row in HQM_df.index:
    for t_period in t_periods:
        HQM_df.loc[row, f'{t_period} Return Percentile'] = ptile(HQM_df[f'{t_period} Price Return'], HQM_df.loc[row, f'{t_period} Price Return'])/100

# Printing the entire DataFrame    
HQM_df

,Symbol,Price,Quantity of Shares to Buy,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile,HQM Score
0,MMM,178.80,NA,0.033379,0.245545,-0.153044,0.156436,-0.109283,0.2,-0.044634,0.463366,NA
1,AOS,84.00,NA,0.517625,0.853465,0.195715,0.831683,0.120902,0.883168,0.052388,0.928713,NA
2,ABT,134.81,NA,0.232279,0.528713,0.206052,0.841584,0.014829,0.611881,0.048656,0.922772,NA
3,ABBV,126.20,NA,0.164770,0.413861,0.085258,0.671287,0.079906,0.805941,0.014847,0.80198,NA
4,ABMD,315.86,NA,0.104620,0.320792,0.040211,0.554455,-0.198809,0.057426,-0.172282,0.057426,NA
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.33,NA,0.206232,0.485149,0.060442,0.59802,-0.042776,0.4,-0.011133,0.671287,NA
501,ZBRA,588.85,NA,0.580647,0.90099,0.145240,0.762376,-0.004040,0.546535,-0.033181,0.534653,NA
502,ZBH,129.86,NA,-0.190522,0.037624,-0.241164,0.055446,-0.176125,0.077228,-0.146554,0.083168,NA
503,ZION,65.85,NA,0.459487,0.80198,0.080987,0.657426,0.092121,0.837624,-0.040920,0.487129,NA


### HQM Score

Calculating a score which incorporates the return on different periods:

In [15]:
from statistics import mean

for row in HQM_df.index:
    momentum_ptiles = []
    for t_period in t_periods:
        momentum_ptiles.append(HQM_df.loc[row, f'{t_period} Return Percentile'])
    HQM_df.loc[row, 'HQM Score'] = mean(momentum_ptiles)  # Finding the mean of each percntile and asssigning it to HQM Score

In [16]:
HQM_df

,Symbol,Price,Quantity of Shares to Buy,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile,HQM Score
0,MMM,178.80,NA,0.033379,0.245545,-0.153044,0.156436,-0.109283,0.2,-0.044634,0.463366,0.266337
1,AOS,84.00,NA,0.517625,0.853465,0.195715,0.831683,0.120902,0.883168,0.052388,0.928713,0.874257
2,ABT,134.81,NA,0.232279,0.528713,0.206052,0.841584,0.014829,0.611881,0.048656,0.922772,0.726238
3,ABBV,126.20,NA,0.164770,0.413861,0.085258,0.671287,0.079906,0.805941,0.014847,0.80198,0.673267
4,ABMD,315.86,NA,0.104620,0.320792,0.040211,0.554455,-0.198809,0.057426,-0.172282,0.057426,0.247525
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.33,NA,0.206232,0.485149,0.060442,0.59802,-0.042776,0.4,-0.011133,0.671287,0.538614
501,ZBRA,588.85,NA,0.580647,0.90099,0.145240,0.762376,-0.004040,0.546535,-0.033181,0.534653,0.686139
502,ZBH,129.86,NA,-0.190522,0.037624,-0.241164,0.055446,-0.176125,0.077228,-0.146554,0.083168,0.063366
503,ZION,65.85,NA,0.459487,0.80198,0.080987,0.657426,0.092121,0.837624,-0.040920,0.487129,0.69604


In [17]:
# Arranging the DataFrame in terms of the descending order of the HQM Score
HQM_df.sort_values(by = 'HQM Score', ascending = False, inplace = True)  

In [18]:
# Extracting the first 50 rows with the highest HQM Scores
HQM_df = HQM_df[:50]
HQM_df

,Symbol,Price,Quantity of Shares to Buy,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile,HQM Score
344,NVDA,302.91,NA,1.300685,0.994059,0.782477,0.99802,0.360512,0.986139,0.032783,0.879208,0.964356
253,INTU,654.44,NA,0.783328,0.968317,0.407982,0.974257,0.151906,0.936634,0.055285,0.936634,0.95396
231,HD,432.80,NA,0.585244,0.90297,0.335359,0.950495,0.243037,0.964356,0.115533,0.988119,0.951485
498,XLNX,219.23,NA,0.492257,0.829703,0.721989,0.992079,0.403169,0.990099,0.095788,0.984158,0.94901
291,LOW,257.83,NA,0.697749,0.946535,0.327114,0.942574,0.211878,0.956436,0.058345,0.944554,0.947525
10,AMD,143.04,NA,0.538806,0.869307,0.771742,0.99604,0.319667,0.978218,0.057800,0.942574,0.946535
236,HPQ,36.96,NA,0.643202,0.928713,0.244423,0.887129,0.268484,0.970297,0.178227,0.99604,0.945545
349,ODFL,359.30,NA,0.743889,0.962376,0.399169,0.972277,0.198266,0.952475,0.030561,0.863366,0.937624
312,MCHP,86.61,NA,1.446747,0.99802,1.213040,1.0,1.184386,1.0,-0.002961,0.712871,0.927723
447,TSCO,230.58,NA,0.742851,0.960396,0.269524,0.908911,0.151574,0.934653,0.039006,0.90297,0.926733


In [19]:
# Resetting the index 
HQM_df.reset_index(drop = True, inplace = True)

In [20]:
# Print the entire DataFrame
HQM_df

,Symbol,Price,Quantity of Shares to Buy,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile,HQM Score
0,NVDA,302.91,NA,1.300685,0.994059,0.782477,0.99802,0.360512,0.986139,0.032783,0.879208,0.964356
1,INTU,654.44,NA,0.783328,0.968317,0.407982,0.974257,0.151906,0.936634,0.055285,0.936634,0.95396
2,HD,432.80,NA,0.585244,0.90297,0.335359,0.950495,0.243037,0.964356,0.115533,0.988119,0.951485
3,XLNX,219.23,NA,0.492257,0.829703,0.721989,0.992079,0.403169,0.990099,0.095788,0.984158,0.94901
4,LOW,257.83,NA,0.697749,0.946535,0.327114,0.942574,0.211878,0.956436,0.058345,0.944554,0.947525
5,AMD,143.04,NA,0.538806,0.869307,0.771742,0.99604,0.319667,0.978218,0.057800,0.942574,0.946535
6,HPQ,36.96,NA,0.643202,0.928713,0.244423,0.887129,0.268484,0.970297,0.178227,0.99604,0.945545
7,ODFL,359.30,NA,0.743889,0.962376,0.399169,0.972277,0.198266,0.952475,0.030561,0.863366,0.937624
8,MCHP,86.61,NA,1.446747,0.99802,1.213040,1.0,1.184386,1.0,-0.002961,0.712871,0.927723
9,TSCO,230.58,NA,0.742851,0.960396,0.269524,0.908911,0.151574,0.934653,0.039006,0.90297,0.926733


### Quantity of shares to buy

In [21]:
# Function for taking input from the user regarding his desired portfolio amount to invest 
def pf_input():
    global pf_size
    pf_size = input("Enter the value of your portfolio:")

    try:
        value = float(pf_size)
    except ValueError:                                       # If the user does not enter the numericals
        print("This is not a number! \n Please try again:")
        pf_size = input("Enter the value of your portfolio:")

pf_input()
print(pf_size)

Enter the value of your portfolio:1000000
1000000


In [22]:
weight = float(pf_size) / len(HQM_df.index)
for i in range(len(HQM_df['Symbol'])-1):
    HQM_df.loc[i, 'Quantity of Shares to Buy'] = math.floor(weight / HQM_df.loc[i, 'Price']) 

HQM_df

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Price,Quantity of Shares to Buy,1-Year Price Return,1-Year Return Percentile,6-Month Price Return,6-Month Return Percentile,3-Month Price Return,3-Month Return Percentile,1-Month Price Return,1-Month Return Percentile,HQM Score
0,NVDA,302.91,66,1.300685,0.994059,0.782477,0.99802,0.360512,0.986139,0.032783,0.879208,0.964356
1,INTU,654.44,30,0.783328,0.968317,0.407982,0.974257,0.151906,0.936634,0.055285,0.936634,0.95396
2,HD,432.80,46,0.585244,0.90297,0.335359,0.950495,0.243037,0.964356,0.115533,0.988119,0.951485
3,XLNX,219.23,91,0.492257,0.829703,0.721989,0.992079,0.403169,0.990099,0.095788,0.984158,0.94901
4,LOW,257.83,77,0.697749,0.946535,0.327114,0.942574,0.211878,0.956436,0.058345,0.944554,0.947525
5,AMD,143.04,139,0.538806,0.869307,0.771742,0.99604,0.319667,0.978218,0.057800,0.942574,0.946535
6,HPQ,36.96,541,0.643202,0.928713,0.244423,0.887129,0.268484,0.970297,0.178227,0.99604,0.945545
7,ODFL,359.30,55,0.743889,0.962376,0.399169,0.972277,0.198266,0.952475,0.030561,0.863366,0.937624
8,MCHP,86.61,230,1.446747,0.99802,1.213040,1.0,1.184386,1.0,-0.002961,0.712871,0.927723
9,TSCO,230.58,86,0.742851,0.960396,0.269524,0.908911,0.151574,0.934653,0.039006,0.90297,0.926733


WE HAVE THE COMPLETE TABULAR DATA WITH ALSO THE QUANTITY OF EACH STOCK TO BE BOUGHT

## Excel Output

Using the XlsxWriter library for Python to create formatted Excel files.

In [23]:
writer = pd.ExcelWriter('Strategy_1.xlsx', engine='xlsxwriter')
HQM_df.to_excel(writer, sheet_name='Quantitative Momentum Strategy', index = False)

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = { 
                    'A': ['Symbol', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Quantity of Shares to Buy', integer_template],
                    'D': ['1-Year Price Return', percent_template],
                    'E': ['1-Year Return Percentile', percent_template],
                    'F': ['6-Month Price Return', percent_template],
                    'G': ['6-Month Return Percentile', percent_template],
                    'H': ['3-Month Price Return', percent_template],
                    'I': ['3-Month Return Percentile', percent_template],
                    'J': ['1-Month Price Return', percent_template],
                    'K': ['1-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Quantitative Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Quantitative Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [26]:
# Saving excel output
writer.save()

# 2) Quantitative Value Strategy

Value investing is an investment strategy that involves picking stocks that appear to be trading for less than their intrinsic or book value. Value investors actively ferret out stocks they think the stock market is underestimating.

We automate this using Python.

We are going to build an investing strategy that selects the 50 stocks with the best value metrics. And then, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

Our metrics comprises of the following indicators:

* <b>Price-to-Earnings(P/E) Ratio:</b> It is the ratio for valuing a company that measures its current share price relative to its earnings per share (EPS). P/E ratios are used by investors and analysts to determine the relative value of a company's shares in an apples-to-apples comparison. It can also be used to compare a company against its own historical record or to compare aggregate markets against one another or over time.

* <b>Price-To-Book(P/B) Ratio:</b> Companies use this ratio to compare a firm's market capitalization to its book value. It's calculated by dividing the company's stock price per share by its book value per share (BVPS). An asset's book value is equal to its carrying value on the balance sheet, and companies calculate it netting the asset against its accumulated depreciation.

* <b>Price-to-Sales(P/S) Ratio:</b> It is a valuation ratio that compares a company’s stock price to its revenues. It is an indicator of the value that financial markets have placed on each dollar of a company’s sales or revenues.

* <b>Enterprise Value(EV):</b> It is a measure of a company's total value, often used as a more comprehensive alternative to equity market capitalization. It includes in its calculation the market capitalization of a company but also short-term and long-term debt as well as any cash on the company's balance sheet.

* <b>EBITDA:</b> Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)

* <b>EV/GP:</b> Enterprise Value / Gross Profit


In [27]:
symbol = 'TSLA'
batch_url_2 = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_API_TOKEN}'
data2 = requests.get(batch_url_2, verify = False).json()
data2

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'TSLA': {'advanced-stats': {'beta': 1.867354894459186,
   'totalCash': 16588964322,
   'currentDebt': 18269379433,
   'revenue': 47709815202,
   'grossProfit': 11166085363,
   'totalRevenue': 47365149741,
   'EBITDA': 7437189782,
   'revenuePerShare': 46.77,
   'revenuePerEmployee': 1007482.26,
   'debtToEquity': 2.15921616359117,
   'profitMargin': 0.07494209033175464,
   'enterpriseValue': 1035119693944,
   'enterpriseValueToRevenue': 21.7,
   'priceToSales': 21.83,
   'priceToBook': 38.42,
   'forwardPERatio': 168.09638045517343,
   'pegRatio': 0.6379692681927758,
   'peHigh': -53.33382953242467,
   'peLow': -318.6030038240607,
   'week52highDate': '2021-10-25',
   'week52lowDate': '2021-03-04',
   'putCallRatio': 1.0223637016443894,
   'companyName': 'Tesla Inc',
   'marketcap': 1040112484108,
   'week52high': 1262.79,
   'week52low': 545.14,
   'week52highSplitAdjustOnly': 1256.92,
   'week52highDateSplitAdjustOnly': '2021-10-28',
   'week52lowSplitAdjustOnly': 552.05,
   'week52

In [28]:
print(data2[symbol]['quote']['peRatio'])
print(data2[symbol]['advanced-stats']['priceToBook'])
print(data2[symbol]['advanced-stats']['priceToSales'])
print(data2[symbol]['advanced-stats']['enterpriseValue'])
print(data2[symbol]['advanced-stats']['EBITDA'])
print(data2[symbol]['advanced-stats']['grossProfit'])

333.53
38.42
21.83
1035119693944
7437189782
11166085363


In [29]:
# P/E Ratio
pe = data2[symbol]['quote']['peRatio']

# P/B Ratio
pb = data2[symbol]['advanced-stats']['priceToBook']

# P/S Ratio
ps = data2[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
ev = data2[symbol]['advanced-stats']['enterpriseValue']
ebitda = data2[symbol]['advanced-stats']['EBITDA']
ev_by_ebitda = ev / ebitda

# EV/GP
gp = data2[symbol]['advanced-stats']['grossProfit']
ev_by_gp = ev / gp

In [30]:
# columns for the DataFrame
vs_cols = [
                'Symbol',
                'Price',
                'Quantity of Shares to Buy', 
                'Price-to-Earnings Ratio',
                'PE Percentile',
                'Price-to-Book Ratio',
                'PB Percentile',
                'Price-to-Sales Ratio',
                'PS Percentile',
                'EV/EBITDA',
                'EV/EBITDA Percentile',
                'EV/GP',
                'EV/GP Percentile',
                'Score'
             ]

### API Call in Batches

In [31]:
vs_df = pd.DataFrame(columns = vs_cols)

for sym_string in sym_strings:
    batch_url_2 = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={sym_string}&types=quote,advanced-stats&token={IEX_API_TOKEN}'
    data_v = requests.get(batch_url_2, verify = False).json()
    
    for symbol in sym_string.split(','):
        ev = data_v[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data_v[symbol]['advanced-stats']['EBITDA']
        gp = data_v[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_by_ebitda = ev/ebitda
        except TypeError:
            ev_by_ebitda = np.NaN
        
        try:
            ev_by_gp = ev / gp
        except TypeError:
            ev_by_gp = np.NaN
            
        vs_df = vs_df.append(
                              pd.Series([
                                        symbol,
                                        data_v[symbol]['quote']['latestPrice'],
                                        'N/A',
                                        data_v[symbol]['quote']['peRatio'],
                                        'N/A',
                                        data_v[symbol]['advanced-stats']['priceToBook'],
                                        'N/A',
                                        data_v[symbol]['advanced-stats']['priceToSales'],
                                        'N/A',
                                        ev_by_ebitda,
                                        'N/A',
                                        ev_by_gp,
                                        'N/A',
                                        'N/A'
                                        ],
                                        index = vs_cols
                                      ),
                              ignore_index = True
                             )

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'sandbox.iexapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/

In [32]:
vs_df

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
0,MMM,184.40,N/A,17.83,N/A,7.23,N/A,2.86,N/A,11.979997,N/A,6.830061,N/A,N/A
1,AOS,85.00,N/A,29.71,N/A,6.98,N/A,4.1,N/A,19.408468,N/A,9.873648,N/A,N/A
2,ABT,133.98,N/A,33.85,N/A,6.93,N/A,5.52,N/A,22.723525,N/A,10.362836,N/A,N/A
3,ABBV,121.81,N/A,30.22,N/A,16.2,N/A,3.9,N/A,10.180458,N/A,7.212786,N/A,N/A
4,ABMD,321.63,N/A,97.74,N/A,9.98,N/A,14.16,N/A,44.881495,N/A,16.249325,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.89,N/A,25.74,N/A,-4.7,N/A,5.85,N/A,20.954854,N/A,9.947566,N/A,N/A
501,ZBRA,595.21,N/A,37.74,N/A,11.42,N/A,5.93,N/A,29.591894,N/A,12.866775,N/A,N/A
502,ZBH,131.37,N/A,32.71,N/A,2.06,N/A,3.23,N/A,17.812576,N/A,5.789613,N/A,N/A
503,ZION,65.55,N/A,6.19,N/A,1.38,N/A,2.79,N/A,4.594301,N/A,2.553595,N/A,N/A


In [33]:
# For missing data in the DataFrame
vs_df[vs_df.isnull().any(axis=1)]

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
24,GOOG,3010.68,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
43,AON,298.48,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
64,BRK.B,295.20,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
149,DISCK,24.36,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
205,FOX,35.01,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
332,NWS,22.89,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
428,TROW,198.67,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
459,UA,20.42,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [34]:
# Filling the missing data with the mean of the other data present in rows of that column 
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    vs_df[column].fillna(vs_df[column].mean(), inplace = True)

In [35]:
vs_df[vs_df.isnull().any(axis=1)]

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score


Now this is empty

### Percentiles

In [36]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in vs_df.index:
    for metric in metrics.keys():
        vs_df.loc[row, metrics[metric]] = ptile(vs_df[metric], vs_df.loc[row, metric])/100

# Printing each percentile score 
for metric in metrics.values():
    print(vs_df[metric])

# Printing the DataFrame    
vs_df

0      0.343564
1       0.59604
2      0.667327
3      0.611881
4      0.954455
         ...   
500    0.528713
501    0.714851
502    0.651485
503    0.089109
504    0.853465
Name: PE Percentile, Length: 505, dtype: object
0      0.736634
1      0.718812
2      0.716832
3       0.90396
4      0.805941
         ...   
500    0.054455
501    0.829703
502    0.288119
503    0.155446
504    0.940594
Name: PB Percentile, Length: 505, dtype: object
0      0.451485
1       0.59703
2      0.712871
3      0.582178
4      0.921782
         ...   
500    0.726733
501    0.728713
502    0.506931
503    0.442574
504    0.921782
Name: PS Percentile, Length: 505, dtype: object
0      0.376238
1      0.659406
2      0.726733
3      0.306931
4      0.940594
         ...   
500     0.69703
501    0.833663
502    0.609901
503    0.055446
504    0.922772
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0      0.443564
1      0.661386
2      0.683168
3      0.489109
4      0.869307
         ...   
5

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
0,MMM,184.40,N/A,17.83,0.343564,7.23,0.736634,2.86,0.451485,11.979997,0.376238,6.830061,0.443564,N/A
1,AOS,85.00,N/A,29.71,0.59604,6.98,0.718812,4.10,0.59703,19.408468,0.659406,9.873648,0.661386,N/A
2,ABT,133.98,N/A,33.85,0.667327,6.93,0.716832,5.52,0.712871,22.723525,0.726733,10.362836,0.683168,N/A
3,ABBV,121.81,N/A,30.22,0.611881,16.20,0.90396,3.90,0.582178,10.180458,0.306931,7.212786,0.489109,N/A
4,ABMD,321.63,N/A,97.74,0.954455,9.98,0.805941,14.16,0.921782,44.881495,0.940594,16.249325,0.869307,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.89,N/A,25.74,0.528713,-4.70,0.054455,5.85,0.726733,20.954854,0.69703,9.947566,0.667327,N/A
501,ZBRA,595.21,N/A,37.74,0.714851,11.42,0.829703,5.93,0.728713,29.591894,0.833663,12.866775,0.772277,N/A
502,ZBH,131.37,N/A,32.71,0.651485,2.06,0.288119,3.23,0.506931,17.812576,0.609901,5.789613,0.378218,N/A
503,ZION,65.55,N/A,6.19,0.089109,1.38,0.155446,2.79,0.442574,4.594301,0.055446,2.553595,0.120792,N/A


### Score

This incorporates all the indicators in one for comparison

In [37]:
for row in vs_df.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(vs_df.loc[row, metrics[metric]])
    vs_df.loc[row, 'Score'] = mean(value_percentiles)
    
vs_df

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
0,MMM,184.40,N/A,17.83,0.343564,7.23,0.736634,2.86,0.451485,11.979997,0.376238,6.830061,0.443564,0.470297
1,AOS,85.00,N/A,29.71,0.59604,6.98,0.718812,4.10,0.59703,19.408468,0.659406,9.873648,0.661386,0.646535
2,ABT,133.98,N/A,33.85,0.667327,6.93,0.716832,5.52,0.712871,22.723525,0.726733,10.362836,0.683168,0.701386
3,ABBV,121.81,N/A,30.22,0.611881,16.20,0.90396,3.90,0.582178,10.180458,0.306931,7.212786,0.489109,0.578812
4,ABMD,321.63,N/A,97.74,0.954455,9.98,0.805941,14.16,0.921782,44.881495,0.940594,16.249325,0.869307,0.898416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,130.89,N/A,25.74,0.528713,-4.70,0.054455,5.85,0.726733,20.954854,0.69703,9.947566,0.667327,0.534851
501,ZBRA,595.21,N/A,37.74,0.714851,11.42,0.829703,5.93,0.728713,29.591894,0.833663,12.866775,0.772277,0.775842
502,ZBH,131.37,N/A,32.71,0.651485,2.06,0.288119,3.23,0.506931,17.812576,0.609901,5.789613,0.378218,0.486931
503,ZION,65.55,N/A,6.19,0.089109,1.38,0.155446,2.79,0.442574,4.594301,0.055446,2.553595,0.120792,0.172673


In [38]:
# Sorting the table by the values in 'Score'
vs_df.sort_values(by = 'Score', inplace = True)

In [39]:
vs_df

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
29,AAL,18.81,N/A,-3.72,0.053465,-1.4600,0.057426,0.4604,0.033663,-7.207079,0.019802,1.392129,0.055446,0.04396
379,PRU,106.80,N/A,5.78,0.083168,0.6582,0.063366,0.5651,0.041584,4.302225,0.047525,0.539861,0.007921,0.048713
206,BEN,33.46,N/A,4.74,0.071287,0.7390,0.077228,0.9623,0.120792,3.005637,0.029703,0.785259,0.015842,0.06297
22,ALL,110.21,N/A,10.53,0.172277,1.2800,0.133663,0.6390,0.047525,2.199537,0.025743,0.644049,0.011881,0.078218
32,AIG,54.83,N/A,8.95,0.132673,0.7137,0.069307,0.9457,0.116832,4.314722,0.049505,0.955881,0.027723,0.079208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,PAYC,432.16,N/A,144.90,0.974257,30.7200,0.954455,26.6000,0.994059,87.832847,0.982178,28.924799,0.972277,0.975446
344,NVDA,305.96,N/A,95.88,0.952475,33.5600,0.960396,32.0200,1.0,78.953242,0.974257,48.696321,0.992079,0.975842
411,NOW,622.91,N/A,576.78,0.99604,34.7200,0.962376,22.3900,0.982178,170.008905,0.99802,27.925997,0.970297,0.981782
171,ENPH,218.74,N/A,198.03,0.982178,46.4200,0.972277,24.3100,0.988119,108.303759,0.992079,55.670042,0.994059,0.985743


In [40]:
# Extracting the first 50 rows with the least values of 'Score'
vs_df = vs_df[:50]
vs_df

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
29,AAL,18.81,N/A,-3.72,0.053465,-1.4600,0.057426,0.4604,0.033663,-7.207079,0.019802,1.392129,0.055446,0.04396
379,PRU,106.80,N/A,5.78,0.083168,0.6582,0.063366,0.5651,0.041584,4.302225,0.047525,0.539861,0.007921,0.048713
206,BEN,33.46,N/A,4.74,0.071287,0.7390,0.077228,0.9623,0.120792,3.005637,0.029703,0.785259,0.015842,0.06297
22,ALL,110.21,N/A,10.53,0.172277,1.2800,0.133663,0.6390,0.047525,2.199537,0.025743,0.644049,0.011881,0.078218
32,AIG,54.83,N/A,8.95,0.132673,0.7137,0.069307,0.9457,0.116832,4.314722,0.049505,0.955881,0.027723,0.079208
309,MET,60.33,N/A,10.15,0.164356,0.7261,0.075248,0.7430,0.077228,4.856531,0.065347,0.708710,0.013861,0.079208
136,DHI,110.07,N/A,4.82,0.073267,1.2700,0.128713,0.6816,0.061386,3.456297,0.033663,2.373568,0.106931,0.080792
71,BA,212.23,N/A,-14.80,0.043564,-8.1300,0.047525,1.9300,0.30495,-31.733759,0.005941,-1293.302490,0.00198,0.080792
353,OGN,29.52,N/A,3.60,0.063366,1.3500,0.149505,0.9030,0.108911,2.467414,0.027723,1.518030,0.067327,0.083366
454,TSN,84.23,N/A,5.00,0.075248,0.8508,0.085149,0.3315,0.017822,3.923603,0.041584,3.489050,0.19802,0.083564


In [41]:
vs_df.reset_index(drop = True, inplace = True)
vs_df

,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
0,AAL,18.81,N/A,-3.72,0.053465,-1.4600,0.057426,0.4604,0.033663,-7.207079,0.019802,1.392129,0.055446,0.04396
1,PRU,106.80,N/A,5.78,0.083168,0.6582,0.063366,0.5651,0.041584,4.302225,0.047525,0.539861,0.007921,0.048713
2,BEN,33.46,N/A,4.74,0.071287,0.7390,0.077228,0.9623,0.120792,3.005637,0.029703,0.785259,0.015842,0.06297
3,ALL,110.21,N/A,10.53,0.172277,1.2800,0.133663,0.6390,0.047525,2.199537,0.025743,0.644049,0.011881,0.078218
4,AIG,54.83,N/A,8.95,0.132673,0.7137,0.069307,0.9457,0.116832,4.314722,0.049505,0.955881,0.027723,0.079208
5,MET,60.33,N/A,10.15,0.164356,0.7261,0.075248,0.7430,0.077228,4.856531,0.065347,0.708710,0.013861,0.079208
6,DHI,110.07,N/A,4.82,0.073267,1.2700,0.128713,0.6816,0.061386,3.456297,0.033663,2.373568,0.106931,0.080792
7,BA,212.23,N/A,-14.80,0.043564,-8.1300,0.047525,1.9300,0.30495,-31.733759,0.005941,-1293.302490,0.00198,0.080792
8,OGN,29.52,N/A,3.60,0.063366,1.3500,0.149505,0.9030,0.108911,2.467414,0.027723,1.518030,0.067327,0.083366
9,TSN,84.23,N/A,5.00,0.075248,0.8508,0.085149,0.3315,0.017822,3.923603,0.041584,3.489050,0.19802,0.083564


### Quantity of shares to buy

In [42]:
weight = float(pf_size) / len(vs_df.index)
for i in range(len(vs_df['Symbol'])-1):
    vs_df.loc[i, 'Quantity of Shares to Buy'] = math.floor(weight / vs_df['Price'][i])
vs_df

/Users/livdea/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Symbol,Price,Quantity of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,Score
0,AAL,18.81,1063,-3.72,0.053465,-1.4600,0.057426,0.4604,0.033663,-7.207079,0.019802,1.392129,0.055446,0.04396
1,PRU,106.80,187,5.78,0.083168,0.6582,0.063366,0.5651,0.041584,4.302225,0.047525,0.539861,0.007921,0.048713
2,BEN,33.46,597,4.74,0.071287,0.7390,0.077228,0.9623,0.120792,3.005637,0.029703,0.785259,0.015842,0.06297
3,ALL,110.21,181,10.53,0.172277,1.2800,0.133663,0.6390,0.047525,2.199537,0.025743,0.644049,0.011881,0.078218
4,AIG,54.83,364,8.95,0.132673,0.7137,0.069307,0.9457,0.116832,4.314722,0.049505,0.955881,0.027723,0.079208
5,MET,60.33,331,10.15,0.164356,0.7261,0.075248,0.7430,0.077228,4.856531,0.065347,0.708710,0.013861,0.079208
6,DHI,110.07,181,4.82,0.073267,1.2700,0.128713,0.6816,0.061386,3.456297,0.033663,2.373568,0.106931,0.080792
7,BA,212.23,94,-14.80,0.043564,-8.1300,0.047525,1.9300,0.30495,-31.733759,0.005941,-1293.302490,0.00198,0.080792
8,OGN,29.52,677,3.60,0.063366,1.3500,0.149505,0.9030,0.108911,2.467414,0.027723,1.518030,0.067327,0.083366
9,TSN,84.23,237,5.00,0.075248,0.8508,0.085149,0.3315,0.017822,3.923603,0.041584,3.489050,0.19802,0.083564


WE HAVE THE COMPLETE TABULAR DATA WITH ALSO THE QUANTITY OF EACH STOCK TO BE BOUGHT

In [43]:
writer = pd.ExcelWriter('Strategy_2.xlsx', engine='xlsxwriter')
HQM_df.to_excel(writer, sheet_name='Qualitative Value Strategy', index = False)

In [44]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [46]:
column_formats = {
                    'A': ['Symbol', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Quantity of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Qualitative Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Qualitative Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [47]:
writer.save()